# Gremlin Workshop - Section 03: Inserting and Updating Data

Before we get too far ahead, we should probably discuss how to insert and update data within Amazon Neptune.  There are also some best practices around loading data transactionally into Neptune in order to provide for high concurrency and throughput.  In this notebook we will cover the steps necessary to insert data, update (or upsert) data, and how to write queries for semi-bulk operations.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## Create a Vertex with Properties

Neptune allows you to supply your own String IDs when creating vertices and edges. You should always try to supply your own IDs – we'll see why this is important in a highly-concurrent write scenario a little later on.

Moreover, you should try wherever possible to use _predictable_ IDs – that is, IDs that you can later on predict when wanting to query for a specific vertex or edge. If you don't supply your own ID, Neptune will create a String-based UUID for you – and you'll be hard pressed to determine the value of this ID when you later want to query for the element.

### 03.01 - Creating a vertex with addV() and property()

Create a new vertex representing yourself, with the ID 'person0'.  Instead of using Lorem Ipsom below, feel free to use your own name or make one up.

Additional documentation for these new steps:
- `addV()` - http://tinkerpop.apache.org/docs/3.4.4/reference/#addvertex-step
- `property()` - http://tinkerpop.apache.org/docs/3.4.4/reference/#addproperty-step

__NOTE: We've also included a traversal at the beginning of this code block with the `drop()` step.  We will cover this in more detail later, but this command is to ensure that the Person vertex with id "person0" does not exist before creating your new "person0" vertex.  Vertices (and edges) in Neptune must have unique IDs.  If you attempt to create a vertex with the same ID, you will get an error.__

In [ ]:
%%gremlin

g.V('person0').drop()

In [ ]:
%%gremlin

g.addV('Person').
    property(id, 'person0').
    property('firstName', 'Lorem').
    property('lastName', 'Ipsom')

### 03.02 - Letting Neptune assign an ID

Next we'll try an example where Neptune will assign an ID for us.  In the following example, create a new vertex representing Sherlock Holmes, with a birth year of 1899. Again, we will let Neptune assign an ID.

In [ ]:
%%gremlin

g.addV('Artist').
    property('name', 'Sherlock Holmes').
    property('birthyear', 1899)

You'll notice that the ID returned by Neptune is in the form of a UUID.  If an ID is not supplied for a new vertex or edge, Neptune will assign a unique UUID.  While this is adventageous in some respects, it is good practice to use an ID native to your dataset.  If your dataset contains a unique identifier for the entities being respresented, choose to use that unique identifier over a UUID.  This will make both the dataset and your application code much easier to read.

### 03.03 - Overwriting property values and cardinality

Actually, we made a mistake. Sherlock Holmes, by most accounts, was born in 1854, not 1899. So we need to correct his birthday.  He also died in 1893, so let's make sure we add that as well.

Before we do that, you should understand how Neptune stores property values.  Be default, Neptune stores all property values as sets.  The Apache TinkerPop framework refers to this as _set cardinality_.  If we were to try using the `property('birthyear',1707)` command here, it would not overwrite the previous value.  It would add the new 1707 value as part of the set.  Instead, we will want to change Neptune's defualt behavior.  You can choose to store property values as _single cardinality_.  See the example below.

See the _Cardinality of Vertex Properties_ and _Updating a Vertex Property_ sections in the [Neptune Gremlin Implementation Differences](https://docs.aws.amazon.com/neptune/latest/userguide/access-graph-gremlin-differences.html) documentation.

Let's first try updating the property without changing the cardinality:

In [ ]:
%%gremlin

g.V().has('name', 'Sherlock Holmes').
    property('birthyear', 1854).
    property('deathyear', 1893)

Let's check our work and see what is returned for birthyear:

In [ ]:
%%gremlin

g.V().has('name','Sherlock Holmes').
    valueMap().with(WithOptions.tokens).unfold()

Notice how birthyear has a set associated with it (includes both the old birth year and new birth year values).  This is due to the defualt nature of Neptune storing property values as sets.  We can change this behavior using the following query to update a property.  Note how the term `single` is now used within the `property()` step to force each property to only store a single value:

In [ ]:
%%gremlin

g.V().has('name', 'Sherlock Holmes').
    property(single, 'birthyear', 1854).
    property(single, 'deathyear', 1893)

Let's check our work one last time...

In [ ]:
%%gremlin

g.V().has('name','Sherlock Holmes').
    valueMap().with(WithOptions.tokens).unfold()

As you can see, both the birth year and death year now have a single value.

### 03.04 Create an Edge Between Two Vertices

Now that we have created a couple of vertices, let's try to create an edge (or relationship) between them.  We do this using the `addE()` or AddEdge step.

Here are some additional resources on how to add edges:

- `addE()` - http://tinkerpop.apache.org/docs/current/reference/#addedge-step
- Other examples -  [Adding an airport (vertex) and a route (edge)](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#_adding_an_airport_vertex_and_a_route_edge).

In the following traversal, we will add a Follows edge between Sherlock Holmes and the vertex representing you:

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Sherlock Holmes').
    addE('follows').from(V('person0'))

The returned result should be the edge ID representing the new edge created between Sherlock Holmes and the vertex representing you.

Let's check our work by querying across the outgoing edge from your vertex:

In [ ]:
%%gremlin

    g.V('person0').out('follows').valueMap().unfold()

You should see all of the properties associated with Sherlock Holmes.

### 03.05 - Conditional Writes

Sometimes, you'll want upsert-like functionality, whereby you create a new vertex (or edge) only if it doesn't already exist. If it already exists, you use the existing instance. 

In Gremlin, you achieve this using a `fold()-coalesce()-unfold()` idiom. For a detailed explanation of this approach, see [Using _coalesce_ to only add a vertex if it does not exist](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#coaladdv).

The `coalesce()` pattern allows you to determine whether an element already exists using any combination of ID, label and property predicates. However, _Neptune does not enforce any label and/or property uniqueness constraints_ – and this can have serious consequences in concurrent write scenarios.

If you are using the `coalesce()` pattern in concurrent write scenarios, but not using _predictable user-supplied IDs_, you can experience situations in which two writer processes (clients, threads, etc) attempt to create-or-return the same element at the same time. If both writers determine the element does not exist and they both attempt to create the element, you may end up with two elements, with the same labels and properties, but different IDs.

Neptune will, however, assert uniqueness for vertex IDs and edge IDs. No two vertices can have the same ID; no two edges can have the same ID (a vertex and an edge, however, can have the same ID).

The general guidance, therefore, is: 

When using `coalesce()`, always identify elements by their ID, rather than a property or label predicate. This is important in highly concurrent write scenarios: using a predictable ID for potentially new elements will ensure that only one writer wins should several writers attempt to create the same element simultaneously.

Additional documentation:

- `coalesce()` - http://tinkerpop.apache.org/docs/3.4.4/reference/#coalesce-step

Let's see this in more detail.  For example, what happens if we were to attempt to create another vertex with ID 'person0' (which is already in our graph)?  Give the following code block a try.

In [ ]:
%%gremlin

g.addV('Person').property(id,'person0')

The error response should state that the `Vertex with id already exists`.

Now let's try the same method, but placing the `addV()` inside of a `fold()-coalesce()-unfold()` pattern.  This will check to see if the vertex is already there.  If it is, the traversal will return the existing vertex id.  If it isn't, it will create the vertex and return its id.

In [ ]:
%%gremlin

g.V('person0').fold().coalesce(
        unfold(),
        addV('Person').property(id,'person0')
    )

Just to keep us honest, let's try adding a new vertex with this type of traversal pattern.  Here we will check for the existance of a vertex with id `person1` first to make sure it does not exist.  Then we will use `fold()-coalesce()-unfold()` pattern to create the vertex:

In [ ]:
%%gremlin

g.V('person1')

In [ ]:
%%gremlin

g.V('person1').
    fold().coalesce(
        unfold(),
        addV('Person').property(id,'person1').property('firstName','John').property('lastName','Doe')
    )

### 03.06 - Bulk write operations

Last, but not least, you can also chain multiple `addV()` and `addE()` steps into a single traversal.  This improves write throughput as each Gremlin query is considered a transaction (by default).  Instead of commiting lots of smaller transactions, we can improve throughput by creating multiple vertices and edges (and properties) in a single transaction.

In the following example, we will create a small sample graph with 5 vertices and 5 edges.  We're also using the `as()` step here.  The `as()` step allows us to save a result in the middle of a traversal and come back to it later in the traversal.  In this example, we're saving the added vertices to be able to reference them again when we create edges.

Further documentation:

- `as()` - http://tinkerpop.apache.org/docs/3.4.4/reference/#as-step
- Best practices for bulk transactional loads with Neptune -  https://github.com/aws-samples/aws-dbs-refarch-graph/tree/master/src/writing-from-amazon-kinesis-data-streams

In [ ]:
%%gremlin

g.addV('Hotel').property('name','MGM').as('MGM').
    addV('Hotel').property('name','Aria').as('Aria').
    addV('Hotel').property('name','Venetian').as('Venetian').
    addV('Hotel').property('name','Wynn').as('Wynn').
    addV('Hotel').property('name','Encore').as('Encore').
    addE('Bus Route').from('MGM').to('Aria').
    addE('Bus Route').from('Aria').to('Venetian').
    addE('Bus Route').from('MGM').to('Wynn').
    addE('Bus Route').from('Encore').to('Aria').
    addE('Bus Route').from('Venetian').to('MGM')

Let's check our work:

In [ ]:
%%gremlin

g.V().hasLabel('Hotel').valueMap()

In [ ]:
%%gremlin

g.E().hasLabel('Bus Route')

### Review

In the previous notebook we covered the many different ways that you can Create and Update data within Neptune using Gremlin.  We also covered some of the best practices regarding how to use vertex/edge IDs and how to batch multiple writes into a single transaction.

### End of Section 03 - Move onto the next notebook.